# 통계 분석 및 가설 검정

- 분산분석 (ANOVA): 세 그룹 이상의 평균이 통계적으로 다른가?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# 데이터 합치기

In [ ]:
df100 = pd.read_csv("/content/전라북도 전주시_음식점품질정보.csv")

In [ ]:
df200 = pd.read_csv("/content/전라북도 전주시_음식점기본정보.csv")

In [ ]:
display(df100, df200)

In [ ]:
df = pd.merge(df100, df200, how='inner')

In [ ]:
df.info()

In [ ]:
# 결측치 삭제

df2 = df.drop(['어워드정보설명', '트립어드바이저평점', '씨트립평점'], axis=1)

In [ ]:
df2.head(6)

In [ ]:
df3 = df2.dropna(subset=['인기도', '네이버평점', '(RTI)지수'])

In [ ]:
df4 = df3.rename(columns={'식당(ID)': 'ID', 
                          '식당명' : 'name',
                          '지역명' : 'city',
                          '(RTI)지수': 'RTI',
                          '온라인화진행여부': 'online', 
                          '수용태세지수': 'acceptance', 
                          '인기도': 'popular', 
                          '네이버평점': 'naver',
                          '식당상태' : 'open',
                          '도로명주소': 'add_new',
                          '지번주소' : 'add_old',
                          '식당위도' : 'la',
                          '식당경도' : 'lo',
                          '식당대표전화번호' : 'phone',
                          '영업신고증업태명' : 'types',
                          '영업인허가명' : 'catetory',
                          '음식점소개내용' : 'intro'})

In [ ]:
df4

In [ ]:
df4.types.value_counts()

In [ ]:
df[df['영업신고증업태명']=='키즈카페']

In [ ]:
list(df4[df4['name']=='한국식당'].intro)

In [ ]:
sorted(df4)

In [ ]:
df4.groupby("types", as_index=False)[["RTI","naver"]].mean().sort_values('naver', ascending=False)

### 분산 분석을 위한 grouping

In [ ]:
korean = df4[df4['types']=='한식']
coffee = df4[df4['types']=='커피숍']
chicken = df4[df4['types']=='호프/통닭']

In [ ]:
#세 그룹의 중앙값 (median)

display(statistics.median(korean['naver']),
statistics.median(coffee['naver']),
statistics.median(chicken['naver']))

# 분산 분석을 위한 가정

<img src = "https://www.statology.org/wp-content/uploads/2021/09/unequal1.png" width = 700>

In [ ]:
# 분산의 동질성 (homogeneity)

stats.levene(korean['naver'], coffee['naver'], chicken['naver'])

# 유의하지 않음 따라서 분산의 동질성이 통계적으로 확인됨

In [ ]:
#분산 - 박스 플롯

fig, ax = plt.subplots(figsize=(10, 5))

ax.boxplot([korean['naver'], coffee['naver'], chicken['naver']])

plt.xticks([1,2,3], ["korean", "coffee", "chicken"])

plt.show()

In [ ]:
import statistics

In [ ]:
statistics.variance(korean['naver'])

In [ ]:
fig, ax = plt.subplots()
sns.distplot(korean['naver'], ax=ax)
sns.distplot(coffee['naver'], ax=ax)
sns.distplot(chicken['naver'], ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(korean['naver'], ax=ax)
sns.kdeplot(coffee['naver'], ax=ax)
sns.kdeplot(chicken['naver'], ax=ax)
plt.show()

In [ ]:
# 정규성 - 정규분포를 따르는가 (normality) - Shapiro-Wilk test

In [ ]:
from scipy.stats import shapiro

In [ ]:
sha, p = shapiro(korean['naver'])
print("shapiro 테스트 결과 계수:", sha, '이며',
      "p-value: ", format(p, '.4f'))

In [ ]:
sha, p = shapiro(coffee['naver'])
print("shapiro 테스트 결과 계수:", sha, '이며',
      "p-value: ", format(p, '.4f'))

In [ ]:
sha, p = shapiro(chicken['naver'])
print("shapiro 테스트 결과 계수:", sha, '이며',
      "p-value: ", format(p, '.4f'))

In [ ]:
# QQ plot

stats.probplot(korean['naver'], dist="norm", plot= plt)
plt.title("Q-Q Plot")

In [ ]:
stats.probplot(coffee['naver'], dist="norm", plot= plt)
plt.title("Q-Q Plot")

In [ ]:
stats.probplot(chicken['naver'], dist="norm", plot= plt)
plt.title("Q-Q Plot")

In [ ]:
# 유의하지 않음 따라서 정규성이 통계적으로 확인되지 않음. ANOVA 대신 Kruskal-Wallis H-Test 필요

# KW H-Test

In [ ]:
stats.kruskal(korean['naver'], coffee['naver'], chicken['naver'])

In [ ]:
kru, p = stats.kruskal(korean['naver'], coffee['naver'], chicken['naver'])
print("KW Test 결과값:", kru, '이며', "p-value: ", format(p, '.4f'))

In [ ]:
#만약 가정을 만족했다면 ANOVA로 다음처럼 수행

anova, p = stats.f_oneway(korean['naver'], coffee['naver'], chicken['naver'])

In [ ]:
print("ANOVA 결과값:", anova, '이며', "p-value: ", format(p, '.4f'))